In [1]:
from langgraph.prebuilt import create_react_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI
import json
import re

In [3]:


# -------------------------------------------------------
# LLM (Single Source of Truth)
# -------------------------------------------------------
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

# -------------------------------------------------------
# Utility: Clean JSON Response
# -------------------------------------------------------
def clean_json(text: str) -> dict:
    cleaned = re.sub(r"```json|```", "", text).strip()
    return json.loads(cleaned)

# -------------------------------------------------------
# TASK STORAGE (Simple List)
# -------------------------------------------------------
task_memory = []

# -------------------------------------------------------
# TOOL 1: Add a subtask to the task list
# -------------------------------------------------------
@tool
def add_task(task_description: str) -> dict:
    """
    Store a single subtask in task memory.
    Input: short text describing a single actionable subtask.
    """
    task_memory.append(task_description)
    return {"status": "added", "task": task_description}

# -------------------------------------------------------
# TOOL 2: Clear all tasks (optional utility)
# -------------------------------------------------------
@tool
def clear_tasks(dummy: str = None) -> dict:
    """
    Clears all tasks from memory.
    Useful when starting a new planning session.
    """
    task_memory.clear()
    return {"status": "cleared", "task_count": 0}

# -------------------------------------------------------
# TOOL 3: Get all tasks (optional utility)
# -------------------------------------------------------
@tool
def get_tasks(dummy: str = None) -> dict:
    """
    Returns all stored subtasks.
    """
    return {"tasks": task_memory, "count": len(task_memory)}

# -------------------------------------------------------
# AGENT (Zero-shot ReAct using LangGraph)
# -------------------------------------------------------
agent = create_react_agent(
    model=llm,
    tools=[add_task, clear_tasks, get_tasks],
)

# -------------------------------------------------------
# USER GOAL
# -------------------------------------------------------
goal = "Plan a week-long trip to Japan focusing on cultural experiences"

prompt = f"""
Your job is to break down the goal into clear, actionable subtasks.

Goal:
{goal}

Rules:
- Use the add_task tool to add EACH subtask.
- Each subtask must be short, specific, and actionable.
- After adding all tasks, call get_tasks to show the final list.
- Do NOT invent additional JSON formats; tool output must remain minimal.
- Think step-by-step before adding tasks.
"""

# -------------------------------------------------------
# RUN AGENT
# -------------------------------------------------------
result = agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": prompt
        }
    ]
})

# -------------------------------------------------------
# FINAL OUTPUT
# -------------------------------------------------------
print("\n🤖 Final Agent Response:\n", result)

print("\n🗂 Task List:")
for i, t in enumerate(task_memory, 1):
    print(f"{i}. {t}")


/var/folders/m8/lr5c6v793qbcszm54dsf6y440000gn/T/ipykernel_12956/1673559162.py:58: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(



🤖 Final Agent Response:
 {'messages': [HumanMessage(content='\nYour job is to break down the goal into clear, actionable subtasks.\n\nGoal:\nPlan a week-long trip to Japan focusing on cultural experiences\n\nRules:\n- Use the add_task tool to add EACH subtask.\n- Each subtask must be short, specific, and actionable.\n- After adding all tasks, call get_tasks to show the final list.\n- Do NOT invent additional JSON formats; tool output must remain minimal.\n- Think step-by-step before adding tasks.\n', additional_kwargs={}, response_metadata={}, id='1f0b503c-3715-4620-bee2-dc6c6d140033'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 213, 'total_tokens': 223, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_na

In [5]:
# -------------------------------------------------------
# TASK PLANNER AGENT — FINAL WORKING VERSION
# -------------------------------------------------------

from langgraph.prebuilt import create_react_agent
from langchain.tools import tool
from langchain_openai import ChatOpenAI
import re, json, os

# -------------------------------------------------------
# LLM (Agent Brain)
# -------------------------------------------------------
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# -------------------------------------------------------
# Helper: Clean JSON (Optional for future JSON-based tools)
# -------------------------------------------------------
def clean_json(text: str) -> dict:
    cleaned = re.sub(r"```json|```", "", text).strip()
    return json.loads(cleaned)

# -------------------------------------------------------
# TASK MEMORY (Environment State)
# -------------------------------------------------------
task_memory = []


# -------------------------------------------------------
# TOOL 1 — Add Task
# -------------------------------------------------------
@tool
def add_task(task_description: str) -> dict:
    """Add a subtask and show BEFORE/AFTER state."""
    
    print("\n🔧 [TOOL CALL] add_task")
    print("➡️ Input:", task_description)
    print("📘 BEFORE:", task_memory)

    task_memory.append(task_description)

    print("📗 AFTER:", task_memory, "\n")

    return {"status": "added", "task": task_description}


# -------------------------------------------------------
# TOOL 2 — Get Tasks
# -------------------------------------------------------
@tool
def get_tasks(dummy: str = None) -> dict:
    """Return all tasks stored so far, with debug output."""
    
    print("\n🔧 [TOOL CALL] get_tasks")
    print("📘 CURRENT TASK MEMORY:", task_memory, "\n")

    return {"tasks": task_memory, "count": len(task_memory)}


# -------------------------------------------------------
# TOOL 3 — Clear Tasks
# -------------------------------------------------------
@tool
def clear_tasks(dummy: str = None) -> dict:
    """Clear all tasks and show BEFORE/AFTER state."""
    
    print("\n🔧 [TOOL CALL] clear_tasks")
    print("📘 BEFORE:", task_memory)

    task_memory.clear()

    print("📗 AFTER:", task_memory, "\n")

    return {"status": "cleared", "task_count": 0}


# -------------------------------------------------------
# AGENT — Zero-Shot ReAct using LangGraph
# -------------------------------------------------------
agent = create_react_agent(
    model=llm,
    tools=[add_task, get_tasks, clear_tasks],
)


# -------------------------------------------------------
# USER GOAL — Try Changing This!
# -------------------------------------------------------
goal = "Organize a weekend team-building event."

prompt = f"""
Break this goal into short, actionable subtasks.
Rules:
- Use `add_task` tool for each subtask.
- Subtasks must be specific and action-oriented.
- After adding all tasks, call `get_tasks`.
- Think step-by-step.
Goal: {goal}
"""


# -------------------------------------------------------
# RUN AGENT
# -------------------------------------------------------
result = agent.invoke({
    "messages": [
        {"role": "user", "content": prompt}
    ]
})

# -------------------------------------------------------
# FINAL OUTPUT
# -------------------------------------------------------
print("\n🤖 FINAL AGENT RESPONSE:\n", result)

print("\n🗂️ FINAL TASK LIST:")
for i, t in enumerate(task_memory, 1):
    print(f"{i}. {t}")


/var/folders/m8/lr5c6v793qbcszm54dsf6y440000gn/T/ipykernel_12956/1848608342.py:83: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(



🔧 [TOOL CALL] add_task
➡️ Input: Determine the budget for the team-building event.
📘 BEFORE: []
📗 AFTER: ['Determine the budget for the team-building event.'] 


🔧 [TOOL CALL] add_task
➡️ Input: Select a date for the weekend team-building event.
📘 BEFORE: ['Determine the budget for the team-building event.']
📗 AFTER: ['Determine the budget for the team-building event.', 'Select a date for the weekend team-building event.'] 


🔧 [TOOL CALL] add_task
➡️ Input: Choose a location for the event.
📘 BEFORE: ['Determine the budget for the team-building event.', 'Select a date for the weekend team-building event.']
📗 AFTER: ['Determine the budget for the team-building event.', 'Select a date for the weekend team-building event.', 'Choose a location for the event.'] 


🔧 [TOOL CALL] add_task
➡️ Input: Plan activities and games for the team-building event.
📘 BEFORE: ['Determine the budget for the team-building event.', 'Select a date for the weekend team-building event.', 'Choose a location for 